### Creation of the environment

In [1]:
%tensorflow_version 2.x
!pip3 install --upgrade pip
#!pip install -qU t5
!pip3 install git+https://github.com/google-research/text-to-text-transfer-transformer.git #extra_id_x support

import functools
import os
import time
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

import tensorflow.compat.v1 as tf
import tensorflow_datasets as tfds

import t5

#Set the base dir(Google cloud bucket)
BASE_DIR = "gs://bucket_code_completion" 

if not BASE_DIR or BASE_DIR == "gs://":
  raise ValueError("You must enter a BASE_DIR.")
ON_CLOUD = True


if ON_CLOUD:
  import tensorflow_gcs_config
  from google.colab import auth
  # Set credentials for GCS reading/writing from Colab and TPU.
  TPU_TOPOLOGY = "2x2"
  try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
    TPU_ADDRESS = tpu.get_master()
    print('Running on TPU:', TPU_ADDRESS)
  except ValueError:
    raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')
  auth.authenticate_user()
  tf.config.experimental_connect_to_host(TPU_ADDRESS)
  tensorflow_gcs_config.configure_gcs_from_colab_auth()

tf.disable_v2_behavior()

# Improve logging.
from contextlib import contextmanager
import logging as py_logging

if ON_CLOUD:
  tf.get_logger().propagate = False
  py_logging.root.setLevel('INFO')

@contextmanager
def tf_verbosity_level(level):
  og_level = tf.logging.get_verbosity()
  tf.logging.set_verbosity(level)
  yield
  tf.logging.set_verbosity(og_level)

     |████████████████████████████████| 1.5MB 6.8MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
  Cloning https://github.com/google-research/text-to-text-transfer-transformer.git to /tmp/pip-req-build-rgg3g6ic
  Running command git clone -q https://github.com/google-research/text-to-text-transfer-transformer.git /tmp/pip-req-build-rgg3g6ic
     |████████████████████████████████| 361 kB 5.1 MB/s 
     |████████████████████████████████| 65 kB 2.8 MB/s 
     |████████████████████████████████| 1.2 MB 9.6 MB/s 
     |████████████████████████████████| 3.4 MB 20.2 MB/s 
     |████████████████████████████████| 3.8 MB 60.0 MB/s 
     |████████████████████████████████| 1.9 MB 70.8 MB/s 
     |████████████████████████████████| 3.2 MB 66.1 MB/s 
     |████████████████████████████████| 883 kB 61.1 MB/s 
  Created wheel for t5: filename=t5-0.9.0-py3-none-any.whl size=230112 sha256=a7554d6768f5f0b077d6892488a15d0ade7d8b427ea155c0

Instructions for updating:
non-resource variables are not supported in the long term


### Path to csv file
This variable contains the path to the tsv file for training loaded on the bucket. Please be sure to insert the correct path

In [2]:
nq_tsv_path = {
    "train":'gs://bucket_code_completion/Python_T5/pretrain_dataset/code.tsv',
    "validation":'gs://bucket_code_completion/Python_T5/pretrain_dataset/code.tsv',
}

### Preprocess of the dataset
In this step we preprocess the dataset.  
You have to change the path to vocab files (*vocab_model_path* and *vocab_path*)


In [3]:
from t5.data import postprocessors as t5_postprocessors
from t5.seqio import Feature,SentencePieceVocabulary


# # Set the path of sentencepiece model and vocab files
vocab_model_path = 'gs://bucket_code_completion/Python_T5/code.model'
vocab_path = 'gs://bucket_code_completion/Python_T5/code.vocab'


TaskRegistry = t5.data.TaskRegistry
TfdsTask = t5.data.TfdsTask


def get_default_vocabulary():
  return SentencePieceVocabulary(vocab_model_path, 100)

DEFAULT_OUTPUT_FEATURES = {
    "inputs": Feature(
        vocabulary=get_default_vocabulary(), add_eos=True, required=False),

    "targets": Feature(
        vocabulary=get_default_vocabulary(), add_eos=True)
}

In [4]:
def nq_dataset_fn(split, shuffle_files=True):
  # We only have one file for each split.
  del shuffle_files

   # Load lines from the text file as examples.

  ds = tf.data.TextLineDataset(nq_tsv_path[split])
  ds = ds.map(
      functools.partial(tf.io.decode_csv, record_defaults=["string","string"],
                        field_delim="\t", use_quote_delim=False),
      num_parallel_calls=tf.data.experimental.AUTOTUNE)
  
  ds = ds.map(lambda *ex: dict(zip(["input", "output"], ex)))
  return ds

print("A few raw train examples...")
for ex in tfds.as_numpy(nq_dataset_fn("train").take(5)):
  print(ex)

A few raw train examples...
{'input': b"def userid<extra_id_0> : <NEW_LINE> <IND> return config [ 'rabbit<extra_id_1> ] [ 'userid' ]<extra_id_2>NEW_LINE> <DE<extra_id_3>>", 'output': b"<extra_id_0> ( )<extra_id_1>'<extra_id_2> <<extra_id_3>IND<extra_id_4>"}
{'input': b'def override_user ( self , username , password = None , homedir<extra_id_0> , <NEW_LINE<extra_id_1> None , msg_login =<extra_id_2> , msg_quit = None ) : <NEW_LINE<extra_id_3> <IND> _<extra_id_4>ase . override_user ( self , username , password<extra_id_5> homedir , perm , <NEW_LINE<extra_id_6> msg_login , msg_quit ) <NEW_LINE><extra_id_7>DE<extra_id_8>', 'output': b'<extra_id_0> = None<extra_id_1>> perm =<extra_id_2> None<extra_id_3>><extra_id_4> \xe2\x81\x87 <extra_id_5> ,<extra_id_6>><extra_id_7> <<extra_id_8>IND><extra_id_9>'}
{'input': b'<extra_id_0> __init__ ( self<extra_id_1> ** options ) : <NEW_LINE> <<extra_id_2>> super ( Lasso<extra_id_3>mlLexer , self )<extra_id_4> __init__ ( \xe2\x81\x87 mlLexer , LassoLexer<ex

In [5]:
def preprocessing(ds):
  
  def to_inputs_and_targets(ex):

        inputs = tf.strings.join([ ex['input']], separator=' ')
        class_label = tf.strings.join([ex['output']], separator=' ')
        return {'inputs': inputs, 'targets': class_label }
    
  return ds.map(to_inputs_and_targets, num_parallel_calls=tf.data.experimental.AUTOTUNE)

In [6]:
#Create a new training task
t5.data.TaskRegistry.remove('pretraining')
t5.data.TaskRegistry.add(
    "pretraining",
    dataset_fn=nq_dataset_fn,
    splits=["train", "validation"],
    text_preprocessor=[preprocessing],
    output_features = DEFAULT_OUTPUT_FEATURES,
    metric_fns=[t5.evaluation.metrics.accuracy],
)

In [7]:
nq_task = t5.data.TaskRegistry.get("pretraining")
ds = nq_task.get_dataset(split="train", sequence_length={"inputs": 256, "targets": 256})
print("A few preprocessed training examples...")
for ex in tfds.as_numpy(ds.take(5)):
  print(ex)

/usr/local/lib/python3.7/dist-packages/t5/seqio/preprocessors.py:65: UserWarning: Creating resources inside a function passed to Dataset.map() is not supported. Create each resource outside the function, and capture it inside the function to use it.
  _tokenize, num_parallel_calls=tf.data.experimental.AUTOTUNE)


A few preprocessed training examples...
{'inputs_pretokenized': b"def __psi1<extra_id_0> (<extra_id_1> , lengthscale , \xe2\x81\x87 <extra_id_2> mu , S ) : <NEW_LINE> <IND> lengthscale<extra_id_3> = np . square (<extra_id_4> )<extra_id_5>_LINE> _psi1_logdenom<extra_id_6> ( S / lengthscale2 + 1. ) . sum<extra_id_7> = - 1 ) <<extra_id_8>_<extra_id_9>><extra_id_10>psi1_log = ( _psi1_logdenom [ : , None ] + np . einsum ( 'nmq,nq-><extra_id_11>' , np .<extra_id_12> ( mu [ : , None , :<extra_id_13> - \xe2\x81\x87 [ None , : , : ] ) , 1. /<extra_id_14> S<extra_id_15> lengthscale2 ) )<extra_id_16> ( - 2. ) <NEW_LINE><extra_id_17>psi1 = variance * np . exp<extra_id_18> _psi<extra_id_19>_<extra_id_20> ) <NEW_LINE> return _psi1 <NEW_LINE> <DEIND>", 'inputs': array([   18,    39, 14592,    63,   739,  1474,     3,    37,     3,
         110,     4,     9,   739,  1474,     3,    37,     3,    63,
           4,    12, 15945,    12,    13,     2, 32097,  2624,    12,
         444,    10,    16,     

### Pretraining of the model
You can pretrain the model running the following two cells.  
Please set the correct path of the variable *MODEL_DIR* (the path to save the pretrained model in) and *PATH_GIN_FILE* (the gin file configuration for the pre-training)

In [8]:
from mesh_tensorflow.transformer.learning_rate_schedules import learning_rate_schedule_noam

#See https://github.com/google-research/text-to-text-transfer-transformer if you want to scale up the model
MODEL_SIZE = "small"  

MODEL_DIR = 'gs://bucket_code_completion/Python_T5/pretrained_with_masking'


model_parallelism, train_batch_size, keep_checkpoint_max = {
    "small": (1, 256, 16),
    "base": (2, 128, 8),
    "large": (8, 64, 4),
    "3B": (8, 16, 1),
    "11B": (8, 16, 1)}[MODEL_SIZE]


tf.io.gfile.makedirs(MODEL_DIR)

model = t5.models.MtfModel(
    model_dir=MODEL_DIR,
    tpu=TPU_ADDRESS,
    tpu_topology=TPU_TOPOLOGY,
    model_parallelism=model_parallelism,
    batch_size=train_batch_size,
    sequence_length={"inputs": 256, "targets": 256},
    learning_rate_schedule = learning_rate_schedule_noam,
    save_checkpoints_steps=5000,
    keep_checkpoint_max=keep_checkpoint_max if ON_CLOUD else None
)

In [ ]:
PATH_GIN_FILE = 'gs://bucket_code_completion/Python_T5/pretrain_config/operative_config.gin'
import gin
with gin.unlock_config():
    gin.parse_config_file(PATH_GIN_FILE)
    TRAIN_STEPS = 400000
    model.train("pretraining", steps=TRAIN_STEPS)

INFO:root:system_path_file_exists:gs://bucket_code_completion/Python_T5/pretrain_config/operative_config.gin
ERROR:root:Path not found: gs://bucket_code_completion/Python_T5/pretrain_config/operative_config.gin


INFO:tensorflow:Using config: {'_model_dir': 'gs://bucket_code_completion/Python_T5/pretrained_with_masking', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': None, '_session_config': graph_options {
  rewrite_options {
    disable_meta_optimizer: true
  }
}
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.26.41.130:8470"
    }
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({'worker': ['10.26.41.130:8470']}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': 'grpc://10.26.41.130:8470', '_evaluation_master': 'grpc://

/usr/local/lib/python3.7/dist-packages/t5/seqio/preprocessors.py:65: UserWarning: Creating resources inside a function passed to Dataset.map() is not supported. Create each resource outside the function, and capture it inside the function to use it.
  _tokenize, num_parallel_calls=tf.data.experimental.AUTOTUNE)


INFO:tensorflow:num_cores_per_replica: 1
INFO:tensorflow:computation_shape: [1, 1, 1, 1]
INFO:tensorflow:num_replicas: 8
INFO:tensorflow:device_assignment.topology.device_coordinates: [[[0 0 0 0]
  [0 0 0 1]
  [1 0 0 0]
  [1 0 0 1]
  [0 1 0 0]
  [0 1 0 1]
  [1 1 0 0]
  [1 1 0 1]]]
INFO:tensorflow:device_assignment.core_assignment: [[[0 0 0 0]]

 [[0 0 0 1]]

 [[1 0 0 0]]

 [[1 0 0 1]]

 [[0 1 0 0]]

 [[0 1 0 1]]

 [[1 1 0 0]]

 [[1 1 0 1]]]
INFO:tensorflow:auto_logical_to_physical_tpu logical_shape=[8] physical_shape=[2, 2, 2]
INFO:tensorflow:auto_logical_to_physical_tpu logical_to_physical = [(0, 0, 0), (0, 0, 1), (0, 1, 0), (0, 1, 1), (1, 1, 0), (1, 1, 1), (1, 0, 0), (1, 0, 1)]
INFO:tensorflow:SimdMeshImpl init: Shape[batch=8] LayoutRules{('heads', 'model'), ('ensemble', 'ensemble'), ('vocab', 'model'), ('experts', 'batch'), ('d_ff', 'model'), ('batch', 'batch')}
INFO:tensorflow:Device Assignment: <tensorflow.python.tpu.device_assignment.DeviceAssignment object at 0x7f417d5b1c10>
INF